In [1]:
from datasets import load_dataset
import json
import os
import torch
from transformers import TextStreamer
from utils import UnslothWrapper
from unsloth import FastLanguageModel

/home/prasanna/coding/transformers-playground/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Skipping import of cpp extensions due to incompatible torch version 2.10.0+cu128 for torchao version 0.15.0             Please see https://github.com/pytorch/ao/issues/2919 for more info
/tmp/ipykernel_81754/3601310835.py:7: UserWarning: WARNING: Unsloth should be imported before [transformers] to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
import torch 

False

In [3]:
OUTPUT_PATH = "data/prasanna_data.json"

In [4]:
MODEL_NAME = "meta-llama/Llama-3.2-1B-Instruct"
NEW_MODEL_NAME = "Prasanna-llama-1B"
MAX_SEQ_LENGTH = 1024
LOAD_IN_4BIT = False

print(f"⏳ Loading {MODEL_NAME}...")
model, tokenizer = UnslothWrapper.load_model_and_tokenizer(
    model_name=MODEL_NAME,
    model_type="language",
    max_seq_length=MAX_SEQ_LENGTH,
    load_in_4bit=LOAD_IN_4BIT,
)


model = UnslothWrapper.get_peft_model(
    model=model,
    r=32,
    lora_alpha=64,
    lora_dropout=0.0,
    bias="none",
    use_gradient_checkpointing="unsloth",
)

# dataset loading 
dataset = load_dataset("json", data_files=OUTPUT_PATH, split="train")
dataset = UnslothWrapper.format_chat_dataset(
    dataset=dataset,
    tokenizer=tokenizer,
    messages_field="messages",
    output_field="text",
    add_generation_prompt=False,
)

bf16 = torch.cuda.is_available() and torch.cuda.is_bf16_supported()
TRAIN_ARGS = {
    "per_device_train_batch_size": 8,
    "gradient_accumulation_steps": 2,
    "warmup_steps": 10,
    "num_train_epochs": 3,
    "learning_rate": 3e-4,
    "weight_decay": 0.01,
    "lr_scheduler_type": "cosine",
    "logging_steps": 5,
    "optim": "adamw_torch",
    "output_dir": "outputs/unsloth-sft",
    "save_strategy": "no",
    "bf16": bf16,
    "fp16": not bf16,
}

trainer = UnslothWrapper.create_sft_trainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    args_kwargs=TRAIN_ARGS,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    dataset_num_proc=2,
    packing=True,
)

print("🚀 Starting Training...")
# UnslothWrapper.train(trainer=trainer) 

⏳ Loading meta-llama/Llama-3.2-1B-Instruct...
==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    NVIDIA GeForce RTX 4050 Laptop GPU. Num GPUs = 1. Max memory: 5.638 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Tokenizing ["text"] (num_proc=4): 100%|██████████| 950/950 [00:01<00:00, 747.43 examples/s] 

🦥 Unsloth: Padding-free auto-enabled, enabling faster training.
🚀 Starting Training...


In [5]:
# @title Hello  
model.save_pretrained("smollLLM-sft-adapters")
tokenizer.save_pretrained("smollLLM-sft-adapters")

('smollLLM-sft-adapters/tokenizer_config.json',
 'smollLLM-sft-adapters/special_tokens_map.json',
 'smollLLM-sft-adapters/chat_template.jinja',
 'smollLLM-sft-adapters/vocab.json',
 'smollLLM-sft-adapters/merges.txt',
 'smollLLM-sft-adapters/added_tokens.json',
 'smollLLM-sft-adapters/tokenizer.json')

In [ ]:
# save model 
QUANTIZATION_8_METHOD = "q8_0"
QUANTIZATION_6_METHOD= "q6_k"
QUANTIZATION_4_METHOD="q4_k_m"
model.save_pretrained_gguf(
    NEW_MODEL_NAME,
    tokenizer,
    quantization_method = QUANTIZATION_4_METHOD
)

In [9]:
print("\n🤖 Training Complete! Running Inference Test...")

# Enable native 2x faster inference
FastLanguageModel.for_inference(model) 

messages = [
    {"role": "system", "content": "You are Prasanna's AI Assistant. You answer questions about his professional background, projects, and skills."},
    {"role": "user", "content": "tell me more about prasanna?"}  ,
    {"role": "user", "content": "does prasanna is good developer?"}  
]

# Prepare inputs
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must be True for generation
    return_tensors = "pt",
).to("cuda") # Use "cpu" if you are not on GPU, but "cuda" is recommended for training

# Generate
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 100, use_cache = True)


🤖 Training Complete! Running Inference Test...
<|im_start|>system
You are Prasanna's AI Assistant. You answer questions about his professional background, projects, and skills.<|im_end|>
<|im_start|>user
tell me more about prasanna?<|im_end|>
<|im_start|>user
does prasanna is good developer?<|im_end|>
<|im_start|>assistant
Yes he is a self-taught engineer who dropped out of college and learned everything on his own through building projects and real-world experience.<|im_end|>


In [ ]:
# print("⏳ Loading Dataset...")
# dataset = load_dataset("json", data_files="data/prasanna_data.json", split="train")

# MODEL_NAME = "HuggingFaceTB/SmolVLM-500M-Instruct"
# MAX_SEQ_LENGTH = 1024 # 135M models don't need massive context
# DTYPE = None 
# LOAD_IN_4BIT = False # 135M is so small, we don't even need 4bit loading!

# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = MODEL_NAME,
#     max_seq_length = MAX_SEQ_LENGTH,
#     dtype = DTYPE,
#     load_in_4bit = LOAD_IN_4BIT,
# )

# def formatting_prompts_func(examples):
#     convos = examples["messages"] 
#     texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
#     return { "text" : texts, }
# dataset = dataset.map(formatting_prompts_func, batched = True,)

# len(dataset)
# for idx in range(len(dataset)):
#     c = formatting_prompts_func({
#         "messages": [dataset[idx]["messages"]]
#     })
#     print(c)

In [1]:
from unsloth import FastLanguageModel
OUTPUT_PATH = "./datasets/prasanna_data.json" 
NEW_MODEL_NAME = "/home/prasanna/coding/transformers-playground/Prasanna-SmolLM-360M"
MAX_SEQ_LENGTH = 1024
LOAD_IN_4BIT = False

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = NEW_MODEL_NAME, # Point to your saved folder
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = None,           # Auto-detect (Float16/Bfloat16)
    load_in_4bit = LOAD_IN_4BIT,    # Match the quantization used during training
)
 
FastLanguageModel.for_inference(model)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/prasanna/coding/transformers-playground/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    NVIDIA GeForce RTX 4050 Laptop GPU. Num GPUs = 1. Max memory: 5.767 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
HuggingFaceTB/SmolLM2-360M-Instruct does not have a padding token! Will use pad_token = <|endoftext|>.


Unsloth 2026.1.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(49152, 960, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=960, out_features=960, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=960, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=960, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
         

In [2]:
from transformers import TextStreamer

messages = [
    {"role": "system", "content": "You are Prasanna's AI Assistant. You answer questions about his professional background, projects, and skills."},
    {"role": "user", "content": "tell me more about prasanna?"}  
    # {"role": "user", "content": "does prasanna is good developer?"}  
]

# Prepare inputs
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must be True for generation
    return_tensors = "pt",
).to("cuda") # Use "cpu" if you are not on GPU, but "cuda" is recommended for training

# Generate
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 100, use_cache = True)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|im_start|>system
You are Prasanna's AI Assistant. You answer questions about his professional background, projects, and skills.<|im_end|>
<|im_start|>user
tell me more about prasanna?<|im_end|>
<|im_start|>assistant
Prasanna is a software engineer who is passionate about AI, AGI, and the future of work. He is based in Chennai, India and has over 3 years of experience building software solutions. He is a self-taught programmer who dropped out of college and learned everything on his own. He is proficient in Java, JavaScript, TypeScript, Rust, Python, and Go. He is based at Highperformr AI where he works in an agile environment with a focus on
